In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random
from datetime import datetime, timedelta

def generate_random_dates():
    current_year = datetime.now().year
    start_date = datetime(current_year, 1, 1) + timedelta(days=random.randint(0, 364))
    end_date = start_date + timedelta(days=random.randint(1, 14))
        
    return  (start_date.month, start_date.day), (end_date.month, end_date.day)

with open(r'Usergrid Issues Summary 929.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    data = [row for row in csv_reader]

data.remove(data[0])

for pack in data:
    res = generate_random_dates()
    pack.append(res[0])
    pack.append(res[1])
    year = 2024
    start_date = datetime(year, res[0][0], res[0][1])
    if res[0][0] == 12 and res[1][0] == 1: year += 1
    end_date = datetime(year, res[1][0], res[1][1])
    print((end_date - start_date).days)
print(data)

2
3
3
11
1
2
7
8
12
14
2
12
5
5
9
5
3
12
12
12
6
12
5
1
2
1
7
12
7
14
3
14
8
10
3
2
2
6
2
3
8
3
10
2
3
12
8
10
2
7
13
14
8
1
7
10
2
1
3
12
8
1
8
1
2
3
9
4
2
4
7
1
13
14
2
1
2
6
2
13
10
11
7
10
4
5
8
2
6
6
6
12
1
3
6
10
10
5
14
14
11
7
8
2
7
10
1
5
2
3
9
11
6
5
4
8
8
5
7
9
9
9
7
6
5
4
10
8
12
4
14
10
1
9
8
7
12
9
13
3
14
9
5
10
1
5
14
13
8
9
9
9
14
13
3
5
13
9
2
2
7
8
3
2
2
1
11
7
14
1
14
6
2
9
3
12
6
8
9
9
10
6
9
1
10
13
3
10
4
9
7
6
14
13
2
14
11
10
8
7
4
5
1
4
8
2
13
2
7
8
12
3
5
1
13
7
11
12
9
9
4
3
4
3
6
3
9
5
5
6
5
10
5
1
8
10
1
1
14
8
7
4
2
10
4
11
9
12
9
11
4
5
5
7
3
14
3
12
10
11
5
4
8
11
7
1
3
11
1
14
12
4
6
7
4
4
9
4
10
8
8
5
3
4
13
13
2
6
14
5
14
13
4
1
6
13
14
8
6
13
2
7
1
6
11
13
3
5
9
10
7
13
1
9
3
11
13
3
3
9
13
2
4
11
13
8
3
6
9
12
5
3
12
8
5
13
12
12
5
7
14
2
2
13
14
14
9
8
7
8
6
10
7
8
6
1
2
10
12
14
10
4
14
14
1
6
4
1
9
10
11
6
10
7
5
2
4
13
4
13
7
4
9
13
1
5
9
1
10
2
11
9
4
6
3
12
6
12
3
5
5
4
1
7
8
14
9
11
4
1
4
4
7
1
14
6
11
5
8
7
3
13
7
4
13
1
5
1
12
12
1
14
11
7

In [2]:
from catboost import CatBoostRegressor, Pool
import pandas as pd

def process_data(raw_data):
    data = []
    for row in raw_data:
        start_date = datetime(2024, row[-2][0], row[-2][1])
        end_date = datetime(2024, row[-1][0], row[-1][1])
        duration = (end_date - start_date).days
        
        processed_row = {
            'feature1': int(row[0]),
            'feature2': int(row[1]),
            'status': row[2],
            'feature4': int(row[3]),
            'task_id': row[4],
            'task_summary_length': len(row[5]),
            'priority': int(row[6]),
            'user_name': row[7],
            'initial_story_point': int(row[8]),
            'current_story_point': int(row[9]),
            'duration': duration
        }
        data.append(processed_row)
    return pd.DataFrame(data)

data_df = process_data(data)
print(data_df)
categorical_features = ['status', 'task_id', 'user_name']
target = 'duration'

     feature1  feature2                    status  feature4        task_id  \
0          23         1                 Completed  12760224   USERGRID-269   
1          23         1                 Completed  12760224   USERGRID-269   
2          23         1                 Completed  12761452   USERGRID-276   
3          23         1                 Completed  12761800   USERGRID-295   
4          23         1                 Completed  12761802   USERGRID-296   
..        ...       ...                       ...       ...            ...   
924        23       226  NotCompletedWithinSprint  12936220  USERGRID-1251   
925        23       226  NotCompletedWithinSprint  12938749  USERGRID-1260   
926        23       226   CompletedInNotherSprint  12777977   USERGRID-423   
927        23       226   CompletedInNotherSprint  12927353  USERGRID-1193   
928        23       226   CompletedInNotherSprint  12939564  USERGRID-1267   

     task_summary_length  priority user_name  initial_story_poi

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Split features and target
X = data_df.drop(columns=[target])
y = data_df[target]

# Train-test split (assuming larger dataset in reality)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize CatBoost Regressor
model = CatBoostRegressor(
    iterations=5000, 
    learning_rate=0.1, 
    depth=6, 
    cat_features=categorical_features, 
    verbose=10,
    task_type="GPU",
    devices='0'  
)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

0:	learn: 55.8669047	total: 20.3ms	remaining: 1m 41s
10:	learn: 54.0347951	total: 181ms	remaining: 1m 22s
20:	learn: 52.6016289	total: 356ms	remaining: 1m 24s
30:	learn: 51.5806562	total: 529ms	remaining: 1m 24s
40:	learn: 50.6372082	total: 686ms	remaining: 1m 22s
50:	learn: 49.0164295	total: 858ms	remaining: 1m 23s
60:	learn: 48.2651067	total: 1.02s	remaining: 1m 22s
70:	learn: 47.6205384	total: 1.17s	remaining: 1m 21s
80:	learn: 46.8321686	total: 1.34s	remaining: 1m 21s
90:	learn: 46.0720495	total: 1.53s	remaining: 1m 22s
100:	learn: 45.0256974	total: 1.71s	remaining: 1m 22s
110:	learn: 44.0237341	total: 1.88s	remaining: 1m 22s
120:	learn: 43.2916589	total: 2.05s	remaining: 1m 22s
130:	learn: 42.7110054	total: 2.22s	remaining: 1m 22s
140:	learn: 42.1302337	total: 2.38s	remaining: 1m 22s
150:	learn: 41.8590775	total: 2.55s	remaining: 1m 21s
160:	learn: 41.5611324	total: 2.72s	remaining: 1m 21s
170:	learn: 40.9253107	total: 2.88s	remaining: 1m 21s
180:	learn: 40.4811988	total: 3.05s	re

In [36]:
# Predict duration for new task
new_task = pd.DataFrame([{
    'feature1': 25,
    'feature2': 2,
    'status': 'Completed',
    'feature4': 13829292,
    'task_id': 'NEW-TASK-101',
    'task_summary_length': 40,
    'priority': 5,
    'user_name': 'greyes',
    'initial_story_point': 0,
    'current_story_point': 8
}])

predicted_duration = model.predict(new_task)
print(f"Predicted Duration: {predicted_duration} days")

Predicted Duration: [25.14519819] days
